# Random forest

Random forests build an ensemble of classifiers, each of which is a tree model constructed using bootstrapped samples from the input data. The results of these models are then combined to yield a single prediction, which, at the expense of some loss in interpretation, have been found to be highly accurate.

Please also refer to the decision tree user documentation for information relevant to the implementation of random forests in MADlib.

This notebook includes impurity importance which was added in 1.15.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


[(u'MADlib version: 1.15-dev, git revision: rc/1.14-rc1-45-g3ab7554, cmake configuration time: Wed Aug  1 18:34:10 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7',)]

# Random forest classification examples

# 1. Load data
Data set related to whether to play golf or not.

In [5]:
%%sql
DROP TABLE IF EXISTS rf_golf CASCADE;

CREATE TABLE rf_golf (
    id integer NOT NULL,
    "OUTLOOK" text,
    temperature double precision,
    humidity double precision,
    "Temp_Humidity" double precision[],
    clouds_airquality text[],
    windy boolean,
    class text
);

INSERT INTO rf_golf VALUES
(1,'sunny', 85, 85, ARRAY[85, 85],ARRAY['none', 'unhealthy'], 'false','Don''t Play'),
(2, 'sunny', 80, 90, ARRAY[80, 90], ARRAY['none', 'moderate'], 'true', 'Don''t Play'),
(3, 'overcast', 83, 78, ARRAY[83, 78], ARRAY['low', 'moderate'], 'false', 'Play'),
(4, 'rain', 70, 96, ARRAY[70, 96], ARRAY['low', 'moderate'], 'false', 'Play'),
(5, 'rain', 68, 80, ARRAY[68, 80], ARRAY['medium', 'good'], 'false', 'Play'),
(6, 'rain', 65, 70, ARRAY[65, 70], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play'),
(7, 'overcast', 64, 65, ARRAY[64, 65], ARRAY['medium', 'moderate'], 'true', 'Play'),
(8, 'sunny', 72, 95, ARRAY[72, 95], ARRAY['high', 'unhealthy'], 'false', 'Don''t Play'),
(9, 'sunny', 69, 70, ARRAY[69, 70], ARRAY['high', 'good'], 'false', 'Play'),
(10, 'rain', 75, 80, ARRAY[75, 80], ARRAY['medium', 'good'], 'false', 'Play'),
(11, 'sunny', 75, 70, ARRAY[75, 70], ARRAY['none', 'good'], 'true', 'Play'),
(12, 'overcast', 72, 90, ARRAY[72, 90], ARRAY['medium', 'moderate'], 'true', 'Play'),
(13, 'overcast', 81, 75, ARRAY[81, 75], ARRAY['medium', 'moderate'], 'false', 'Play'),
(14, 'rain', 71, 80, ARRAY[71, 80], ARRAY['low', 'unhealthy'], 'true', 'Don''t Play');

SELECT * FROM rf_golf ORDER BY id;

Done.
Done.
14 rows affected.
14 rows affected.


[(1, u'sunny', 85.0, 85.0, [85.0, 85.0], [u'none', u'unhealthy'], False, u"Don't Play"),
 (2, u'sunny', 80.0, 90.0, [80.0, 90.0], [u'none', u'moderate'], True, u"Don't Play"),
 (3, u'overcast', 83.0, 78.0, [83.0, 78.0], [u'low', u'moderate'], False, u'Play'),
 (4, u'rain', 70.0, 96.0, [70.0, 96.0], [u'low', u'moderate'], False, u'Play'),
 (5, u'rain', 68.0, 80.0, [68.0, 80.0], [u'medium', u'good'], False, u'Play'),
 (6, u'rain', 65.0, 70.0, [65.0, 70.0], [u'low', u'unhealthy'], True, u"Don't Play"),
 (7, u'overcast', 64.0, 65.0, [64.0, 65.0], [u'medium', u'moderate'], True, u'Play'),
 (8, u'sunny', 72.0, 95.0, [72.0, 95.0], [u'high', u'unhealthy'], False, u"Don't Play"),
 (9, u'sunny', 69.0, 70.0, [69.0, 70.0], [u'high', u'good'], False, u'Play'),
 (10, u'rain', 75.0, 80.0, [75.0, 80.0], [u'medium', u'good'], False, u'Play'),
 (11, u'sunny', 75.0, 70.0, [75.0, 70.0], [u'none', u'good'], True, u'Play'),
 (12, u'overcast', 72.0, 90.0, [72.0, 90.0], [u'medium', u'moderate'], True, u'Play'),
 (13, u'overcast', 81.0, 75.0, [81.0, 75.0], [u'medium', u'moderate'], False, u'Play'),
 (14, u'rain', 71.0, 80.0, [71.0, 80.0], [u'low', u'unhealthy'], True, u"Don't Play")]

# 2.  Train random forest
Train random forest and view the summary table:

In [6]:
%%sql
DROP TABLE IF EXISTS train_output, train_output_group, train_output_summary;

SELECT madlib.forest_train('rf_golf',         -- source table
                           'train_output',    -- output model table
                           'id',              -- id column
                           'class',           -- response
                           '"OUTLOOK", temperature, humidity, windy',   -- features
                           NULL,              -- exclude columns
                           NULL,              -- grouping columns
                           20::integer,       -- number of trees
                           2::integer,        -- number of random features
                           TRUE::boolean,     -- variable importance
                           1::integer,        -- num_permutations
                           8::integer,        -- max depth
                           3::integer,        -- min split
                           1::integer,        -- min bucket
                           10::integer        -- number of splits per continuous variable
                           );

SELECT * FROM train_output_summary;

Done.
1 rows affected.
1 rows affected.


[(u'forest_train', True, u'rf_golf', u'train_output', u'id', u'class', u'"OUTLOOK",windy,temperature,humidity', u'"OUTLOOK",windy', u'temperature,humidity', None, 20, 2, 8, 3, 1, 10, False, True, 1, 1, 0, 14, 0, u'"Don\'t Play","Play"', u'text', u'text, boolean, double precision, double precision', u'None')]

Review the group table for variable importance:

In [7]:
%%sql
SELECT * FROM train_output_group;

1 rows affected.


[(1, True, [3, 2], [u'overcast', u'sunny', u'rain', u'False', u'True'], Decimal('0.28571428571428571429'), [0.06625, 0.0233928571428572, 0.0, 0.0145833333333333], [0.27116534244345, 0.10438655409181, 0.307256009757555, 0.184889160052295])]

The 'cat_levels_in_text' array shows the levels of the categorical variables "OUTLOOK" and windy, which have 3 and 2 levels respectively. 

Out-of-bag and impurity variable importance arrays are ordered according to the order of the variables in 'independent_varnames' in model summary table. A higher value means higher importance for the variable. 

We can use the helper function to get a normalized view of variable importance:

In [9]:
%%sql
DROP TABLE IF EXISTS imp_output;

SELECT madlib.get_var_importance('train_output','imp_output');
SELECT * FROM imp_output ORDER BY oob_var_importance DESC;

Done.
1 rows affected.
4 rows affected.


[(u'"OUTLOOK"', 63.5636778983438, 31.2511535374489),
 (u'windy', 22.4443175328384, 12.0302992992133),
 (u'humidity', 13.9920045688178, 21.3080310195216),
 (u'temperature', 0.0, 35.4105161438162)]

# 3. Predict
Predict output categories.  For the purpose of this example, we use the same data that was used for training:

In [10]:
%%sql
DROP TABLE IF EXISTS prediction_results;

SELECT madlib.forest_predict('train_output',        -- tree model    
                             'rf_golf',             -- new data table
                             'prediction_results',  -- output table
                             'response');           -- show response

SELECT g.id, class, estimated_class FROM prediction_results p, 
rf_golf g WHERE p.id = g.id ORDER BY g.id;

Done.
1 rows affected.
14 rows affected.


[(1, u"Don't Play", u"Don't Play"),
 (2, u"Don't Play", u"Don't Play"),
 (3, u'Play', u'Play'),
 (4, u'Play', u'Play'),
 (5, u'Play', u'Play'),
 (6, u"Don't Play", u"Don't Play"),
 (7, u'Play', u'Play'),
 (8, u"Don't Play", u"Don't Play"),
 (9, u'Play', u'Play'),
 (10, u'Play', u'Play'),
 (11, u'Play', u'Play'),
 (12, u'Play', u'Play'),
 (13, u'Play', u'Play'),
 (14, u"Don't Play", u"Don't Play")]

To display the probabilities associated with each value of the dependent variable, set the 'type' parameter to 'prob':

In [11]:
%%sql
DROP TABLE IF EXISTS prediction_results;

SELECT madlib.forest_predict('train_output',        -- tree model    
                             'rf_golf',             -- new data table
                             'prediction_results',  -- output table
                             'prob');               -- show probability

SELECT g.id, class, "estimated_prob_Don't Play",  "estimated_prob_Play" 
FROM prediction_results p, rf_golf g WHERE p.id = g.id ORDER BY g.id;

Done.
1 rows affected.
14 rows affected.


[(1, u"Don't Play", 0.95, 0.05),
 (2, u"Don't Play", 0.9, 0.1),
 (3, u'Play', 0.05, 0.95),
 (4, u'Play', 0.25, 0.75),
 (5, u'Play', 0.2, 0.8),
 (6, u"Don't Play", 0.7, 0.3),
 (7, u'Play', 0.1, 0.9),
 (8, u"Don't Play", 0.8, 0.2),
 (9, u'Play', 0.05, 0.95),
 (10, u'Play', 0.3, 0.7),
 (11, u'Play', 0.15, 0.85),
 (12, u'Play', 0.4, 0.6),
 (13, u'Play', 0.05, 0.95),
 (14, u"Don't Play", 0.85, 0.15)]

# 4. View tree in text format
View a single tree within the forest identified by 'gid' and 'sample_id', out of the several that were created:

In [9]:
%%sql
SELECT madlib.get_tree('train_output',1,7, FALSE);

1 rows affected.


[(u'-------------------------------------\n    - Each node represented by \'id\' inside ().\n    - Each internal nodes has the split condition at the end, while each\n        leaf node has a * at the end.\n    - For each internal node (i), its child nodes are indented by 1 level\n        with ids (2i+1) for True node and (2i+2) for False node.\n    - Number of (weighted) rows for each response variable inside [].\'\n        The response label order is given as [\'"Don\\\'t Play"\', \'"Play"\'].\n        For each leaf, the prediction is given after the \'-->\'\n        \n-------------------------------------\n(0)[ 4 12]  temperature <= 65\n   (1)[2 0]  * --> "Don\'t Play"\n   (2)[ 2 12]  humidity <= 78\n      (5)[0 8]  * --> "Play"\n      (6)[2 4]  windy in {False}\n         (13)[1 4]  temperature <= 70\n            (27)[0 3]  * --> "Play"\n            (28)[1 1]  * --> "Don\'t Play"\n         (14)[1 0]  * --> "Don\'t Play"\n',)]

Please see the decision tree user docs for an explanation on how to interpret the tree display above.

# 5. View tree in dot format

In [10]:
%%sql
SELECT madlib.get_tree('train_output',1,7);

1 rows affected.


[(u'digraph "Classification tree for rf_golf" {\n"0" [label="temperature <= 65", shape=ellipse];\n"0" -> "1"[label="yes"];\n"1" [label="\\"Don\'t Play\\"",shape=box];\n"0" -> "2"[label="no"];\n"2" [label="humidity <= 78", shape=ellipse];\n"2" -> "5"[label="yes"];\n"5" [label="\\"Play\\"",shape=box];\n"2" -> "6"[label="no"];\n"6" [label="windy <= False", shape=ellipse];\n"6" -> "13"[label="yes"];\n"6" -> "14"[label="no"];\n"14" [label="\\"Don\'t Play\\"",shape=box];\n"13" [label="temperature <= 70", shape=ellipse];\n"13" -> "27"[label="yes"];\n"27" [label="\\"Play\\"",shape=box];\n"13" -> "28"[label="no"];\n"28" [label="\\"Don\'t Play\\"",shape=box];\n\n} //---end of digraph--------- ',)]

# 6. View tree in dot format with additional information

In [11]:
%%sql
SELECT madlib.get_tree('train_output',1,7, TRUE, TRUE);

1 rows affected.


[(u'digraph "Classification tree for rf_golf" {\n"0" [label="temperature <= 65\\n impurity = 0.375\\n samples = 16\\n value = [ 4 12]\\n class = \\"Play\\"", shape=ellipse];\n"0" -> "1"[label="yes"];\n"1" [label="\\"Don\'t Play\\"\\n impurity = 0\\n samples = 2\\n value = [2 0]",shape=box];\n"0" -> "2"[label="no"];\n"2" [label="humidity <= 78\\n impurity = 0.244898\\n samples = 14\\n value = [ 2 12]\\n class = \\"Play\\"", shape=ellipse];\n"2" -> "5"[label="yes"];\n"5" [label="\\"Play\\"\\n impurity = 0\\n samples = 8\\n value = [0 8]",shape=box];\n"2" -> "6"[label="no"];\n"6" [label="windy <= False\\n impurity = 0.444444\\n samples = 6\\n value = [2 4]\\n class = \\"Play\\"", shape=ellipse];\n"6" -> "13"[label="yes"];\n"6" -> "14"[label="no"];\n"14" [label="\\"Don\'t Play\\"\\n impurity = 0\\n samples = 1\\n value = [1 0]",shape=box];\n"13" [label="temperature <= 70\\n impurity = 0.32\\n samples = 5\\n value = [1 4]\\n class = \\"Play\\"", shape=ellipse];\n"13" -> "27"[label="yes"];\n"27" [label="\\"Play\\"\\n impurity = 0\\n samples = 3\\n value = [0 3]",shape=box];\n"13" -> "28"[label="no"];\n"28" [label="\\"Don\'t Play\\"\\n impurity = 0.5\\n samples = 2\\n value = [1 1]",shape=box];\n\n} //---end of digraph--------- ',)]

You may wish to visualize the tree using pygraphviz or another program that can handle dot format:

In [12]:
# set paths for your environment
from os.path import expanduser
home = expanduser("/Users/fmcquillan/Documents/Product/MADlib/Demos/1 dot 14 demos")

dot_output = %sql select madlib.get_tree('train_output',1,7, TRUE, TRUE);
with open('tree_out.dot', 'w') as f: 
     f.write(dot_output[0][0])
    
import pygraphviz as pgv
from IPython.display import Image
graph = pgv.AGraph("tree_out.dot")
print bool(graph)
graph.draw('tree_out.png',prog='dot')
Image('tree_out.png')

1 rows affected.
True


Please see the decision tree user docs for an explanation on an important difference about how categorical variable tests are displayed in text and dot format.

# 7. Arrays of features
Categorical and continuous features can be array columns, in which case the array is expanded to treat each element of the array as a feature.

In [12]:
%%sql
DROP TABLE IF EXISTS train_output, train_output_group, train_output_summary;

SELECT madlib.forest_train('rf_golf',         -- source table
                           'train_output',    -- output model table
                           'id',              -- id column
                           'class',           -- response
                           '"Temp_Humidity", clouds_airquality',   -- features
                           NULL,              -- exclude columns
                           NULL,              -- grouping columns
                           20::integer,       -- number of trees
                           2::integer,        -- number of random features
                           TRUE::boolean,     -- variable importance
                           1::integer,        -- num_permutations
                           8::integer,        -- max depth
                           3::integer,        -- min split
                           1::integer,        -- min bucket
                           10::integer        -- number of splits per continuous variable
                           );

SELECT * FROM train_output_summary;

Done.
1 rows affected.
1 rows affected.


[(u'forest_train', True, u'rf_golf', u'train_output', u'id', u'class', u'(clouds_airquality)[1],(clouds_airquality)[2],("Temp_Humidity")[1],("Temp_Humidity")[2]', u'(clouds_airquality)[1],(clouds_airquality)[2]', u'("Temp_Humidity")[1],("Temp_Humidity")[2]', None, 20, 2, 8, 3, 1, 10, False, True, 1, 1, 0, 14, 0, u'"Don\'t Play","Play"', u'text', u'text, text, double precision, double precision', u'None')]

# 8. Sample ratio
Use the sample ratio parameter to train on a subset of the data:

In [13]:
%%sql
DROP TABLE IF EXISTS train_output, train_output_group, train_output_summary;

SELECT madlib.forest_train('rf_golf',         -- source table
                           'train_output',    -- output model table
                           'id',              -- id column
                           'class',           -- response
                           '"OUTLOOK", temperature, humidity, windy',   -- features
                           NULL,              -- exclude columns
                           NULL,              -- grouping columns
                           20::integer,       -- number of trees
                           2::integer,        -- number of random features
                           TRUE::boolean,     -- variable importance
                           1::integer,        -- num_permutations
                           8::integer,        -- max depth
                           3::integer,        -- min split
                           1::integer,        -- min bucket
                           10::integer,       -- number of splits per continuous variable
                           NULL,              -- NULL handling
                           FALSE,             -- Verbose
                           0.5                -- Sample ratio
                           );

SELECT * FROM train_output_group;

Done.
1 rows affected.
1 rows affected.


[(1, True, [3, 2], [u'overcast', u'sunny', u'rain', u'False', u'True'], Decimal('0.64285714285714285714'), [0.0, 0.015625, 0.015625, 0.0208333333333333], [0.04719444444445, 0.060156462585, 0.11203471740695, 0.103798888701435])]

Notice the tree is different than a previous one trained without sampling.

In [16]:
# set paths for your environment
from os.path import expanduser
home = expanduser("/Users/fmcquillan/Documents/Product/MADlib/Demos/1 dot 14 demos")

dot_output = %sql select madlib.get_tree('train_output',1,11, TRUE, TRUE);
with open('tree_out.dot', 'w') as f: 
     f.write(dot_output[0][0])
    
import pygraphviz as pgv
from IPython.display import Image
graph = pgv.AGraph("tree_out.dot")
print bool(graph)
graph.draw('tree_out.png',prog='dot')
Image('tree_out.png')

1 rows affected.
True


# Random forest regression examples

# 1. Load data
Data set related to fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models). Data was extracted from the 1974 Motor Trend US magazine.

In [14]:
%%sql
DROP TABLE IF EXISTS mt_cars;

CREATE TABLE mt_cars (
    id integer NOT NULL,
    mpg double precision,
    cyl integer,
    disp double precision,
    hp integer,
    drat double precision,
    wt double precision,
    qsec double precision,
    vs integer,
    am integer,
    gear integer,
    carb integer
);

INSERT INTO mt_cars VALUES
(1,18.7,8,360,175,3.15,3.44,17.02,0,0,3,2),
(2,21,6,160,110,3.9,2.62,16.46,0,1,4,4),
(3,24.4,4,146.7,62,3.69,3.19,20,1,0,4,2),
(4,21,6,160,110,3.9,2.875,17.02,0,1,4,4),
(5,17.8,6,167.6,123,3.92,3.44,18.9,1,0,4,4),
(6,16.4,8,275.8,180,3.078,4.07,17.4,0,0,3,3),
(7,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1),
(8,17.3,8,275.8,180,3.078,3.73,17.6,0,0,3,3),
(9,21.4,null,258,110,3.08,3.215,19.44,1,0,3,1),
(10,15.2,8,275.8,180,3.078,3.78,18,0,0,3,3),
(11,18.1,6,225,105,2.768,3.46,20.22,1,0,3,1),
(12,32.4,4,78.7,66,4.08,2.20,19.47,1,1,4,1),
(13,14.3,8,360,245,3.21,3.578,15.84,0,0,3,4),
(14,22.8,4,140.8,95,3.92,3.15,22.9,1,0,4,2),
(15,30.4,4,75.7,52,4.93,1.615,18.52,1,1,4,2),
(16,19.2,6,167.6,123,3.92,3.44,18.3,1,0,4,4),
(17,33.9,4,71.14,65,4.22,1.835,19.9,1,1,4,1),
(18,15.2,null,304,150,3.15,3.435,17.3,0,0,3,2),
(19,10.4,8,472,205,2.93,5.25,17.98,0,0,3,4),
(20,27.3,4,79,66,4.08,1.935,18.9,1,1,4,1),
(21,10.4,8,460,215,3,5.424,17.82,0,0,3,4),
(22,26,4,120.3,91,4.43,2.14,16.7,0,1,5,2),
(23,14.7,8,440,230,3.23,5.345,17.42,0,0,3,4),
(24,30.4,4,95.14,113,3.77,1.513,16.9,1,1,5,2),
(25,21.5,4,120.1,97,3.70,2.465,20.01,1,0,3,1),
(26,15.8,8,351,264,4.22,3.17,14.5,0,1,5,4),
(27,15.5,8,318,150,2.768,3.52,16.87,0,0,3,2),
(28,15,8,301,335,3.54,3.578,14.6,0,1,5,8),
(29,13.3,8,350,245,3.73,3.84,15.41,0,0,3,4),
(30,19.2,8,400,175,3.08,3.845,17.05,0,0,3,2),
(31,19.7,6,145,175,3.62,2.77,15.5,0,1,5,6),
(32,21.4,4,121,109,4.11,2.78,18.6,1,1,4,2);

SELECT * FROM mt_cars ORDER BY am;

Done.
Done.
32 rows affected.
32 rows affected.


[(29, 13.3, 8, 350.0, 245, 3.73, 3.84, 15.41, 0, 0, 3, 4),
 (27, 15.5, 8, 318.0, 150, 2.768, 3.52, 16.87, 0, 0, 3, 2),
 (25, 21.5, 4, 120.1, 97, 3.7, 2.465, 20.01, 1, 0, 3, 1),
 (9, 21.4, None, 258.0, 110, 3.08, 3.215, 19.44, 1, 0, 3, 1),
 (11, 18.1, 6, 225.0, 105, 2.768, 3.46, 20.22, 1, 0, 3, 1),
 (13, 14.3, 8, 360.0, 245, 3.21, 3.578, 15.84, 0, 0, 3, 4),
 (3, 24.4, 4, 146.7, 62, 3.69, 3.19, 20.0, 1, 0, 4, 2),
 (5, 17.8, 6, 167.6, 123, 3.92, 3.44, 18.9, 1, 0, 4, 4),
 (19, 10.4, 8, 472.0, 205, 2.93, 5.25, 17.98, 0, 0, 3, 4),
 (21, 10.4, 8, 460.0, 215, 3.0, 5.424, 17.82, 0, 0, 3, 4),
 (23, 14.7, 8, 440.0, 230, 3.23, 5.345, 17.42, 0, 0, 3, 4),
 (1, 18.7, 8, 360.0, 175, 3.15, 3.44, 17.02, 0, 0, 3, 2),
 (30, 19.2, 8, 400.0, 175, 3.08, 3.845, 17.05, 0, 0, 3, 2),
 (16, 19.2, 6, 167.6, 123, 3.92, 3.44, 18.3, 1, 0, 4, 4),
 (14, 22.8, 4, 140.8, 95, 3.92, 3.15, 22.9, 1, 0, 4, 2),
 (6, 16.4, 8, 275.8, 180, 3.078, 4.07, 17.4, 0, 0, 3, 3),
 (10, 15.2, 8, 275.8, 180, 3.078, 3.78, 18.0, 0, 0, 3, 3),
 (8, 17.3, 8, 275.8, 180, 3.078, 3.73, 17.6, 0, 0, 3, 3),
 (18, 15.2, None, 304.0, 150, 3.15, 3.435, 17.3, 0, 0, 3, 2),
 (32, 21.4, 4, 121.0, 109, 4.11, 2.78, 18.6, 1, 1, 4, 2),
 (26, 15.8, 8, 351.0, 264, 4.22, 3.17, 14.5, 0, 1, 5, 4),
 (24, 30.4, 4, 95.14, 113, 3.77, 1.513, 16.9, 1, 1, 5, 2),
 (22, 26.0, 4, 120.3, 91, 4.43, 2.14, 16.7, 0, 1, 5, 2),
 (20, 27.3, 4, 79.0, 66, 4.08, 1.935, 18.9, 1, 1, 4, 1),
 (12, 32.4, 4, 78.7, 66, 4.08, 2.2, 19.47, 1, 1, 4, 1),
 (4, 21.0, 6, 160.0, 110, 3.9, 2.875, 17.02, 0, 1, 4, 4),
 (2, 21.0, 6, 160.0, 110, 3.9, 2.62, 16.46, 0, 1, 4, 4),
 (28, 15.0, 8, 301.0, 335, 3.54, 3.578, 14.6, 0, 1, 5, 8),
 (17, 33.9, 4, 71.14, 65, 4.22, 1.835, 19.9, 1, 1, 4, 1),
 (15, 30.4, 4, 75.7, 52, 4.93, 1.615, 18.52, 1, 1, 4, 2),
 (7, 22.8, 4, 108.0, 93, 3.85, 2.32, 18.61, 1, 1, 4, 1),
 (31, 19.7, 6, 145.0, 175, 3.62, 2.77, 15.5, 0, 1, 5, 6)]

# 2.  Train random forest with grouping and surrogates
We train a regression random forest tree with grouping on transmission type (0 = automatic, 1 = manual) and use surrogates for NULL handling:

In [15]:
%%sql
DROP TABLE IF EXISTS mt_cars_output, mt_cars_output_group, mt_cars_output_summary;

SELECT madlib.forest_train('mt_cars',         -- source table
                           'mt_cars_output',  -- output model table
                           'id',              -- id column
                           'mpg',             -- response
                           '*',               -- features
                           'id, hp, drat, am, gear, carb',  -- exclude columns
                           'am',              -- grouping columns
                           10::integer,       -- number of trees
                           2::integer,        -- number of random features
                           TRUE::boolean,     -- variable importance
                           1,                 -- num_permutations
                           10,                -- max depth
                           8,                 -- min split
                           3,                 -- min bucket
                           10,                -- number of splits per continuous variable
                           'max_surrogates=2' -- NULL handling
                           );

SELECT * FROM mt_cars_output_summary;

Done.
1 rows affected.
1 rows affected.


[(u'forest_train', False, u'mt_cars', u'mt_cars_output', u'id', u'mpg', u'vs,cyl,disp,qsec,wt', u'vs,cyl', u'disp,qsec,wt', u'am', 10, 2, 10, 8, 3, 10, False, True, 1, 2, 0, 32, 0, None, u'double precision', u'integer, integer, double precision, double precision, double precision', u'None')]

Review the group table to see variable importance by group:

In [16]:
%%sql
SELECT * FROM mt_cars_output_group ORDER BY gid;

2 rows affected.


[(1, 0, True, [2, 3], [u'0', u'1', u'4', u'6', u'8'], 9.02518813678994, [3.27437852678571, 5.11869678113842, 3.456005, 1.69090166666667, 2.64476081632653], [3.34409180528, 1.180789912157, 4.0143746624229, 1.987039338848, 2.595222156443]),
 (2, 1, True, [2, 3], [u'0', u'1', u'4', u'6', u'8'], 12.3234806918696, [7.67188583333333, 11.7029212868481, 15.69645390625, 0.0, 18.2606219444444], [7.93109211433, 9.112014009263, 14.112089342763, 2.836969504704, 11.291365053425])]

Use the helper function to display normalized variable importance:

In [17]:
%%sql
DROP TABLE IF EXISTS mt_imp_output;

SELECT madlib.get_var_importance('mt_cars_output','mt_imp_output');
SELECT * FROM mt_imp_output ORDER BY am, oob_var_importance DESC;

Done.
1 rows affected.
10 rows affected.


[(0, u'cyl', 31.6266798136018, 8.99888201496216),
 (0, u'disp', 21.3534749649495, 30.5938284017064),
 (0, u'vs', 20.2312669968611, 25.4855561460076),
 (0, u'wt', 16.3410741245189, 19.7783684870616),
 (0, u'qsec', 10.4475041000687, 15.1433649502623),
 (1, u'wt', 34.239597267579, 24.9348163610914),
 (1, u'disp', 29.4316514472623, 31.1638455198447),
 (1, u'cyl', 21.9435741528927, 20.1221371309527),
 (1, u'vs', 14.3851771322661, 17.5142973837102),
 (1, u'qsec', 0.0, 6.26490360440106)]

# 3. Predict

Predict regression output for the same data and compare with original:

In [18]:
%%sql
DROP TABLE IF EXISTS prediction_results;

SELECT madlib.forest_predict('mt_cars_output',
                             'mt_cars',
                             'prediction_results',
                             'response');

SELECT s.am, s.id, mpg, estimated_mpg, mpg-estimated_mpg as delta FROM prediction_results p, 
mt_cars s WHERE s.id = p.id ORDER BY s.am, s.id;

Done.
1 rows affected.
32 rows affected.


[(0, 1, 18.7, 15.8031832298137, 2.89681677018633),
 (0, 3, 24.4, 19.6109880952381, 4.7890119047619),
 (0, 5, 17.8, 18.8089047619048, -1.00890476190476),
 (0, 6, 16.4, 15.0550760869565, 1.34492391304348),
 (0, 8, 17.3, 14.7450760869565, 2.55492391304348),
 (0, 9, 21.4, 18.4258975155279, 2.97410248447205),
 (0, 10, 15.2, 14.1930760869565, 1.00692391304348),
 (0, 11, 18.1, 17.9839047619048, 0.116095238095241),
 (0, 13, 14.3, 15.5301832298137, -1.23018322981366),
 (0, 14, 22.8, 20.2247380952381, 2.57526190476191),
 (0, 16, 19.2, 18.8089047619048, 0.391095238095236),
 (0, 18, 15.2, 15.8031832298137, -0.603183229813665),
 (0, 19, 10.4, 14.0221832298137, -3.62218322981367),
 (0, 21, 10.4, 14.0221832298137, -3.62218322981366),
 (0, 23, 14.7, 14.8841832298137, -0.184183229813664),
 (0, 25, 21.5, 20.2247380952381, 1.2752619047619),
 (0, 27, 15.5, 15.5301832298137, -0.0301832298136624),
 (0, 29, 13.3, 15.5301832298137, -2.23018322981366),
 (0, 30, 19.2, 15.5301832298137, 3.66981677018634),
 (1, 2, 21.0, 20.8864007936508, 0.113599206349203),
 (1, 4, 21.0, 21.1089007936508, -0.108900793650793),
 (1, 7, 22.8, 23.7355555555556, -0.935555555555556),
 (1, 12, 32.4, 29.1780555555556, 3.22194444444445),
 (1, 15, 30.4, 29.9038055555556, 0.496194444444448),
 (1, 17, 33.9, 29.1780555555556, 4.72194444444444),
 (1, 20, 27.3, 29.1780555555556, -1.87805555555555),
 (1, 22, 26.0, 25.6858888888889, 0.31411111111111),
 (1, 24, 30.4, 29.9038055555556, 0.496194444444441),
 (1, 26, 15.8, 19.7780674603175, -3.97806746031746),
 (1, 28, 15.0, 19.7780674603175, -4.77806746031746),
 (1, 31, 19.7, 20.8864007936508, -1.18640079365079),
 (1, 32, 21.4, 23.3028888888889, -1.90288888888889)]

# 4.  View tree

In [22]:
%%sql
SELECT madlib.get_tree('mt_cars_output',1,7);

1 rows affected.


[(u'digraph "Regression tree for mt_cars" {\n"0" [label="disp <= 258", shape=ellipse];\n"0" -> "1"[label="yes"];\n"0" -> "2"[label="no"];\n"1" [label="disp <= 146.7", shape=ellipse];\n"1" -> "3"[label="yes"];\n"3" [label="23.3833",shape=box];\n"1" -> "4"[label="no"];\n"4" [label="19.3143",shape=box];\n"2" [label="wt <= 3.44", shape=ellipse];\n"2" -> "5"[label="yes"];\n"5" [label="17.3",shape=box];\n"2" -> "6"[label="no"];\n"6" [label="13.72",shape=box];\n\n} //---end of digraph--------- ',)]

In [24]:
# set paths for your environment
from os.path import expanduser
home = expanduser("/Users/fmcquillan/Documents/Product/MADlib/Demos/1 dot 14 demos")

dot_output = %sql select madlib.get_tree('mt_cars_output',1,7, TRUE, TRUE);
with open('tree_out.dot', 'w') as f: 
     f.write(dot_output[0][0])
    
import pygraphviz as pgv
from IPython.display import Image
graph = pgv.AGraph("tree_out.dot")
print bool(graph)
graph.draw('tree_out.png',prog='dot')
Image('tree_out.png')

1 rows affected.
True


Display the surrogate variables that are used to compute the split for each node when the primary variable is NULL:

In [25]:
%%sql
SELECT madlib.get_tree_surr('mt_cars_output',1,7);

1 rows affected.


[(u'\n  -------------------------------------\n      Surrogates for internal nodes\n  -------------------------------------\n        \n(0) disp <= 258\n     1: vs in {1}    [common rows = 23]\n     2: qsec > 17.42    [common rows = 22]\n     [Majority branch = 13 ]\n\n(1) disp <= 146.7\n     1: wt <= 3.19    [common rows = 13]\n     2: cyl in {4}    [common rows = 11]\n     [Majority branch = 7 ]\n\n(2) wt <= 3.44\n     1: qsec <= 17.3    [common rows = 8]\n     2: vs in {0}    [common rows = 5]\n     [Majority branch = 5 ]\n\n',)]

# NULL handling example

# 1. Load data
Create toy example to illustrate null-as-category handling for categorical features:

In [19]:
%%sql
DROP TABLE IF EXISTS null_handling_example;

CREATE TABLE null_handling_example (
    id integer,
    country text,
    city text,
    weather text,
    response text
);
 
INSERT INTO null_handling_example VALUES
(1,null,null,null,'a'),
(2,'US',null,null,'b'),
(3,'US','NY',null,'c'),
(4,'US','NY','rainy','d');
 
SELECT * FROM null_handling_example ORDER BY id;

Done.
Done.
4 rows affected.
4 rows affected.


[(1, None, None, None, u'a'),
 (2, u'US', None, None, u'b'),
 (3, u'US', u'NY', None, u'c'),
 (4, u'US', u'NY', u'rainy', u'd')]

# 2. Train random forest
Note that '__NULL__' is set as a valid level for the categorical features country, weather and city:

In [20]:
%%sql
DROP TABLE IF EXISTS train_output, train_output_group, train_output_summary;

SELECT madlib.forest_train('null_handling_example',  -- source table
                           'train_output',    -- output model table
                           'id',              -- id column
                           'response',        -- response
                           'country, weather, city',   -- features
                           NULL,              -- exclude columns
                           NULL,              -- grouping columns
                           10::integer,        -- number of trees
                           2::integer,        -- number of random features
                           TRUE::boolean,     -- variable importance
                           1::integer,        -- num_permutations
                           3::integer,        -- max depth
                           2::integer,        -- min split
                           1::integer,        -- min bucket
                           3::integer,        -- number of splits per continuous variable
                           'null_as_category=TRUE'
                           );

SELECT * FROM train_output_summary;

Done.
1 rows affected.
1 rows affected.


[(u'forest_train', True, u'null_handling_example', u'train_output', u'id', u'response', u'country,weather,city', u'country,weather,city', u'', None, 10, 2, 3, 2, 1, 3, False, True, 1, 1, 0, 4, 0, u'"a","b","c","d"', u'text', u'text, text, text', u'__NULL__')]

In [21]:
%%sql
SELECT * FROM train_output_group;

1 rows affected.


[(1, True, [2, 2, 2], [u'US', u'__NULL__', u'rainy', u'__NULL__', u'NY', u'__NULL__'], Decimal('1.00000000000000000000'), [0.0, 0.0, 0.0], [0.125, 0.0944444444444, 0.1836666666667])]

# 3. Predict

Predict for data not previously seen by assuming NULL value as the default.  

In [22]:
%%sql
DROP TABLE IF EXISTS table_test;

CREATE TABLE table_test (
    id integer,
    country text,
    city text,
    weather text,
    expected_response text
);

INSERT INTO table_test VALUES
(1,'IN','MUM','cloudy','a'),
(2,'US','HOU','humid','b'),
(3,'US','NY','sunny','c'),
(4,'US','NY','rainy','d');
 
DROP TABLE IF EXISTS prediction_results;

SELECT madlib.forest_predict('train_output',
                             'table_test',
                             'prediction_results',
                             'response');

SELECT s.id, expected_response, estimated_response
FROM prediction_results p, table_test s
WHERE s.id = p.id ORDER BY id;

Done.
Done.
4 rows affected.
Done.
1 rows affected.
4 rows affected.


[(1, u'a', u'a'), (2, u'b', u'a'), (3, u'c', u'c'), (4, u'd', u'c')]

There is only training data for country 'US' so the response for country 'IN' is 'a' as expected, corresponding to a NULL (not 'US') country level.  

Likewise, any city in the 'US' that is not 'NY' will predict response 'b' as expected, corresponding to a NULL (not 'NY') city level. 